In [32]:
import qiskit
import numpy as np
from qiskit import QuantumCircuit , execute , Aer, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram , plot_bloch_multivector
from math import sqrt, pi

q = QuantumRegister(4)
c = ClassicalRegister(3)
ckt = QuantumCircuit(q, c)
backend = Aer.get_backend('qasm_simulator')
shots = 10

N = 3
theta = np.linspace(0, pi, N)
phi = np.linspace(0, pi/2, N)
# maximum entanglement
r = pi/2

strategies = [(i, j) for i in theta for j in phi]
payoff_matrix=np.array(
    [[0,0,0],[-9,-9,-1],[-9,1,-9],[1,9,9],[1,-9,-9],[9,1,9],[9,9,1],[2,2,2]]
            )

strategy_pair = {}
payoff_pair = {}
for theta1, phi1 in strategies:
    U1 = np.array([[np.exp(phi1*1j)*np.cos(theta1/2),np.sin(theta1/2)],[-np.sin(theta1/2),np.exp(-phi1*1j)*np.cos(theta1/2)]])
    payoff_max = [0, 0]
    for theta2, phi2 in strategies:
        U2 = np.array([[np.exp(phi2*1j)*np.cos(theta2/2),np.sin(theta2/2)],[-np.sin(theta2/2),np.exp(-phi2*1j)*np.cos(theta2/2)]])
        for theta3, phi3 in strategies:
            U3 = np.array([[np.exp(phi3*1j)*np.cos(theta3/2),np.sin(theta3/2)],[-np.sin(theta3/2),np.exp(-phi3*1j)*np.cos(theta3/2)]])
            # clear the quantum circuit
            ckt.data.clear()
            ckt.reset(q)
            # end
            ckt.cx(q[3],q[2])
            ckt.cx(q[3],q[1])
            ckt.cx(q[3],q[0])
            ckt.barrier()
            ckt.cx(q[1],q[2])
            ckt.cx(q[1],q[0])
            ckt.rx(-np.pi/2,1)
            ckt.cx(q[1],q[2])
            ckt.cx(q[1],q[0])
            ckt.barrier()
            ckt.squ(U1, q[0])
            ckt.squ(U2, q[1])
            ckt.squ(U3, q[2])
            ckt.barrier()
            ckt.cx(q[1],q[0])
            ckt.cx(q[1],q[2])
            ckt.rx(np.pi/2,1)
            ckt.cx(q[1],q[0])
            ckt.cx(q[1],q[2])
            ckt.barrier()
            ckt.measure([0,1,2], [0,1,2])
            results = execute(ckt, backend=backend, shots=shots).result()
            answer = results.get_counts()
            payoff = [0, 0, 0]
            #order in answer need to reverse
            for i in answer:
                if (i=="111"):
                    payoff=payoff+payoff_matrix[7]*answer[i]/shots
                if (i=="110"):
                    payoff=payoff+payoff_matrix[3]*answer[i]/shots
                if (i=="101"):
                    payoff=payoff+payoff_matrix[5]*answer[i]/shots
                if (i=="100"):
                    payoff=payoff+payoff_matrix[1]*answer[i]/shots
                if (i=="011"):
                    payoff=payoff+payoff_matrix[6]*answer[i]/shots
                if (i=="010"):
                    payoff=payoff+payoff_matrix[2]*answer[i]/shots
                if (i=="001"):
                    payoff=payoff+payoff_matrix[4]*answer[i]/shots
                if (i=="000"):
                    payoff=payoff+payoff_matrix[0]*answer[i]/shots
            if payoff[1] >= payoff_max[0] and payoff[2] >= payoff_max[1]:
                strategy_pair[(theta1, phi1)] = ((theta2, phi2), (theta3, phi3))
                payoff_pair[(theta1, phi1)] = payoff
                payoff_max[0] = payoff[1]
                payoff_max[1] = payoff[2]

# NE
# symmetric
NE = [(k, v) for (k, v) in strategy_pair.items() if strategy_pair[v[0]] == (k, v[1]) and strategy_pair[v[1]] == (k, v[0])]
if any(NE):
    for cnt, ne in enumerate(NE):
        print(f'******NE #{cnt+1}******')
        print(f'Player 1: theta = {ne[0][0]}, phi = {ne[0][1]}')
        print(f'Player 2: theta = {ne[1][0][0]}, phi = {ne[1][0][1]}')
        print(f'Player 3: theta = {ne[1][1][0]}, phi = {ne[1][1][1]}')
        print('Payoff: ', payoff_pair[ne[0]])
else:
    print('No Nash Equilibrium is found.')

#print(NE)

******NE #1******
Player 1: theta = 0.0, phi = 1.5707963267948966
Player 2: theta = 0.0, phi = 1.5707963267948966
Player 3: theta = 0.0, phi = 1.5707963267948966
Payoff:  [2. 2. 2.]
